## Import Cora and Facebook/Page-Page daasets

In [1]:
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import FacebookPagePage

dataset_cora = Planetoid(root="../data", name="Cora")
dataset_facebook = FacebookPagePage(root="../data/Facebook-Page-Page")

data_cora = dataset_cora[0]
data_facebook = dataset_facebook[0]

# Unlike Cora, Facebook Page-Page doesn’t have training, evaluation, and test masks by
# default. We can arbitrarily create masks with the range() function:
data_facebook.train_mask = range(18000)
data_facebook.val_mask = range(18001, 20000)
data_facebook.test_mask = range(20001, 22470)

print(data_cora)
print(data_facebook)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Data(x=[22470, 128], edge_index=[2, 342004], y=[22470], train_mask=[18000], val_mask=[1999], test_mask=[2469])


Cora is a popular dataset for node classification in the scientific literature. It represents a network of 2,708 publications (**nodes**), where each connection is a reference (**edge**). Each publication is described as a binary vector of 1,433 unique words (**node features**), where 0 and 1 indicate the absence or presence of the corresponding word, respectively. Each node has a class label from 7 **classes**.  

## Classifying nodes with vanilla neural networks

We will consider node features (i.e., 1,433) as a regular tabular dataset. We will train a simple neural
network (MLP) on this dataset to classify our nodes. Note that this architecture does not take into account
the topology of the network. We will try to fix this issue with a vanilla GNN and GCN, then compare the results.

In [2]:
import pandas as pd

# This is just to show the feature matrix and class vector (we need torch.Tensor type input --> data_cora.x)
df_x = pd.DataFrame(data_cora.x.numpy()) # 1,433 columns
df_x['label'] = pd.DataFrame(data_cora.y) # adding a label-column
df_x

,0,1,2,3,4,5,6,7,8,9,...,1424,1425,1426,1427,1428,1429,1430,1431,1432,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2704,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [3]:
import torch
from torch.nn import Linear
import torch.nn.functional as F

In [5]:
class MLP(torch.nn.Module):
    def __init__(self, dim_input, dim_hidden, dim_output):
        super().__init__()
        self.linear1 = Linear(dim_input, dim_hidden)
        self.linear2 = Linear(dim_hidden, dim_output)

    def forward(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        return F.log_softmax(x, dim=1)

model_cora = MLP(dataset_cora.num_features, 16, dataset_cora.num_classes)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)
model_cora

MLP(
  (linear1): Linear(in_features=1433, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=7, bias=True)
)

In [25]:
input = data_cora.x[:1,:]
print(input.shape)
print(model_cora(input))
print(model_cora(input).shape)
print(model_cora(input).argmax(dim=1))

torch.Size([1, 1433])
tensor([[-1.9166, -2.1035, -1.9224, -2.0546, -1.8943, -1.7017, -2.0902]],
       grad_fn=<LogSoftmaxBackward0>)
torch.Size([1, 7])
tensor([5])


In [11]:
# define a simple accuracy metric (not the best metric for multiclass classification) 
def accuracy(y_pred, y_true):
    return torch.sum(y_pred == y_true) / len(y_true)
    
criterion = torch.nn.CrossEntropyLoss() # loss
optimizer = torch.optim.Adam(model_cora.parameters(), lr=0.005, weight_decay=5e-4)
num_epochs = 100

In [26]:
# Train MLP model on Cora dataset
model_cora.train()
for epoch in range(num_epochs+1):
    optimizer.zero_grad()
    y_pred = model_cora(data_cora.x)
    loss = criterion(y_pred[data_cora.train_mask], data_cora.y[data_cora.train_mask]) # loss = criterion(y_pred, y_true)
    acc = accuracy(y_pred[data_cora.train_mask].argmax(dim=1), data_cora.y[data_cora.train_mask])
    loss.backward()
    optimizer.step()
    if (epoch % 20) == 0:
        val_loss = criterion(y_pred[data_cora.val_mask], data_cora.y[data_cora.val_mask])
        val_acc = accuracy(y_pred[data_cora.val_mask].argmax(dim=1), data_cora.y[data_cora.val_mask])
        print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: {acc*100:>5.2f}%'
              f'| Val Loss: {val_loss:.2f} | Val Acc: {val_acc*100:.2f}%')

Epoch   0 | Train Loss: 1.960 | Train Acc: 14.29%| Val Loss: 1.96 | Val Acc: 11.40%
Epoch  20 | Train Loss: 0.556 | Train Acc: 100.00%| Val Loss: 1.55 | Val Acc: 48.60%
Epoch  40 | Train Loss: 0.094 | Train Acc: 100.00%| Val Loss: 1.40 | Val Acc: 51.60%
Epoch  60 | Train Loss: 0.037 | Train Acc: 100.00%| Val Loss: 1.37 | Val Acc: 54.20%
Epoch  80 | Train Loss: 0.022 | Train Acc: 100.00%| Val Loss: 1.37 | Val Acc: 54.00%
Epoch 100 | Train Loss: 0.017 | Train Acc: 100.00%| Val Loss: 1.37 | Val Acc: 54.00%


In [27]:
# Test MLP model on Cora dataset    
model_cora.eval()
y_pred = model_cora(data_cora.x)
acc = accuracy(y_pred[data_cora.test_mask].argmax(dim=1), data_cora.y[data_cora.test_mask])
print(f'\nMLP test accuracy for Cora dataset: {acc*100:.2f}%')


MLP test accuracy for Cora dataset: 54.10%


## Repeat the process for the Facebook Page-Page dataset

In [28]:
model_fb = MLP(dataset_facebook.num_features, 16, dataset_facebook.num_classes)
model_fb

MLP(
  (linear1): Linear(in_features=128, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=4, bias=True)
)

In [32]:
# Train MLP model on Facebook Page-Page dataset
optimizer = torch.optim.Adam(model_fb.parameters(), lr=0.005, weight_decay=5e-4)
model_fb.train()
for epoch in range(num_epochs+1):
    optimizer.zero_grad()
    y_pred = model_fb(data_facebook.x)
    train_loss = criterion(y_pred[data_facebook.train_mask], data_facebook.y[data_facebook.train_mask])
    train_acc = accuracy(y_pred[data_facebook.train_mask].argmax(dim=1), data_facebook.y[data_facebook.train_mask])
    train_loss.backward()
    optimizer.step()
    if (epoch % 20) == 0:
        val_loss = criterion(y_pred[data_facebook.val_mask], data_facebook.y[data_facebook.val_mask])
        val_acc = accuracy(y_pred[data_facebook.val_mask].argmax(dim=1), data_facebook.y[data_facebook.val_mask])
        print(f'Epoch {epoch:>3} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:>5.2f}%'
              f'| Val Loss: {val_loss:.2f} | Val Acc: {val_acc*100:.2f}%')

Epoch   0 | Train Loss: 1.416 | Train Acc: 25.76%| Val Loss: 1.41 | Val Acc: 25.16%
Epoch  20 | Train Loss: 0.832 | Train Acc: 68.37%| Val Loss: 0.84 | Val Acc: 66.68%
Epoch  40 | Train Loss: 0.641 | Train Acc: 74.78%| Val Loss: 0.66 | Val Acc: 73.74%
Epoch  60 | Train Loss: 0.589 | Train Acc: 76.50%| Val Loss: 0.62 | Val Acc: 74.74%
Epoch  80 | Train Loss: 0.566 | Train Acc: 77.34%| Val Loss: 0.61 | Val Acc: 76.04%
Epoch 100 | Train Loss: 0.552 | Train Acc: 77.88%| Val Loss: 0.60 | Val Acc: 75.89%


In [33]:
# Test MLP model on Facebook Page-Page dataset   
model_fb.eval()
y_pred = model_fb(data_facebook.x)
test_acc = accuracy(y_pred[data_facebook.test_mask].argmax(dim=1), data_facebook.y[data_facebook.test_mask])
print(f'\nMLP test accuracy for Facebook Page-Page dataset: {test_acc*100:.2f}%')


MLP test accuracy for Facebook Page-Page dataset: 74.44%


## Classifying nodes with vanilla graph neural networks